# Deep learning

Deep learning refers to highly multi-layer neural networks that have lots of parameters. Training them can be slow, so be prepared to wait if you have a low-end PC. 

Let's have a look at some popular frameworks for deep learning. The most popular is [tensorflow](https://www.tensorflow.org/), that allows one to create complex computing graphs in Python, while the actual heavy lifting is done by underlying C++ implementations. While tensorflow itself is really powerful, we'll be having a look at [keras](https://keras.io/), an abstraction on top of tensorflow that lets you define neural network in an easy manner.

If you're new to Jupyter notebooks, you can run a cell by clicking on it and pressing `ctrl` + `enter`. The variables, functions etc in previous cells will remain in memory, so you can refer to them later in other cells.

This exercise has some ready-made code and you should implement the parts where is says #TODO. Feel free to create your own functions or classes as well.

**First things first, download the [HASYv2](https://zenodo.org/record/259444#.WcZjIZ8xDCI) dataset into the same folder as this notebook, and extract it.**

In [1]:
# Python2 compatibility
from __future__ import print_function

import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.utils import shuffle

/home/osboxes/venv/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)



Implement a function that reads the HASYv2 dataset from the given folder. It should return a tuple containing two numpy arrays: the data as a `(68233, 32, 32)` tensor, and a size `168233` vector containing the labels ("symbol_id") as integers.
Hint. Use scipy's imread to read the `.png` images as black-and-white. Each image should be a 32x32 matrix.

In [2]:
from scipy.misc import imread

# TODO 
def read_data(folder):
    # Helper function to read symbols into numpy array
    def read_symbols(dataframe):
        return dataframe['symbol_id'].values # values returns numpy array
    
    # Helper function to read images into numpy matrix
    def read_images(dataframe):
        imgs = np.zeros((len(dataframe), 32, 32)) # size of the whole image set
        i = 0
        for idx, row in dataframe.iterrows():
            path = '../data/HASYv2/' + row['path']
            img = imread(path, flatten=False, mode='L')
            imgs[i] = img
            i = i + 1
        return imgs
    
    # Init return symbols
    images, symbols = None, None    
    df = pd.read_csv(folder + 'hasy-data-labels.csv')
    symbols = read_symbols(df)
    images = read_images(df)
    # Return the values
    return images, symbols

# two numpy arrays
X, y = read_data("../data/HASYv2/")

# Assert the input

print(X.shape, y.shape) # Should be (168233, 32, 32) (168233,)

(168233, 32, 32) (168233,)


Overfitting is when we fit the model to work well on our training data, but . Fitting a model to predict the training set perfectly is in most cases trivial, and is a simple exercise in [optimization](https://en.wikipedia.org/wiki/Mathematical_optimization). In machine learning, however, we are interested in _generalization error_, or in other words, how well our model works on previously unseen data. Therefore, we want to evaluate the performance of our model on data that it has not yet seen: this is a way to approximate how well it generalizes. 

As a reminder from last week, tuning the hyperparameters of your model or choosing your model based on the test data **overfits to the test data**. We don't want this, so we use a so-called _validation set_ to tune our model.

To do this, **split the data into training, validation and test sets. Be sure to shuffle it first, otherwise your validation and test sets will not contain most labels**. Your function should return a tuple containing the training, validation and test data, i.e `(X_train, y_train, X_val, y_val, X_test, y_test)`. You can use e.g the proportions `0.8, 0.1, 0.1`.

In [3]:
# TODO
def split_data(X, y, perc=[0.8, 0.1, 0.1]):
    test_s = perc[1]
    #val_s = 1.0/((1 - test_s)*10)
    val_s = 0.111104 # better value for same output
    # Use sklearn function
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_s)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_s)
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y)

print(X_train.shape, y_train.shape) # Should yield approx (134586, 32, 32) (134586,)

(134586, 32, 32) (134586,)


In [4]:
all_uniques = list(np.unique(y))
uniques_renumbered = [x for x in range(len(all_uniques))]

Since there's 369 different classes with overall over 150000 images, let's reduce the complexity of our task by taking only the 100 first classes. Also note that the `symbol_id` field does not start from 0, but instead has arbitrary numbers. 

**Transform the labels so that the numbering for the class goes from 0 to 99, and discard the rest of the classes and corresponding images.**

In [5]:
# Give new ids to classes such that the first unique symbol_id gets the number 0,
# the next unique symbol_id gets the number 1, and so forth
# TODO
def transform_labels(y):
    # y: symbol array (numpy)
    for i in range(len(y)):
        index_of_unique = all_uniques.index(y[i])
        y[i] = uniques_renumbered[index_of_unique]
    return y

y_train, y_val, y_test = map(transform_labels, [y_train, y_val, y_test])

print(y_train.shape, y_val.shape, y_test.shape) # Should be approx (134586,) (16823,) (16824,)

(134586,) (16823,) (16824,)


In [6]:
# Should return the elements in arr for which their corresponding label in y_arr is in between [0, 100]
# TODO
def filter_out(arr, y_arr):
    good_indices = np.isin(y_arr, [x for x in range(100)])
    # return the tuple
    return arr[good_indices], y_arr[good_indices]

X_train, y_train = filter_out(X_train, y_train)
X_val, y_val = filter_out(X_val, y_val)
X_test, y_test = filter_out(X_test, y_test)

print(y_train.shape, X_train.shape) # Should be approx (34062,) (34062, 32, 32)

(34004,) (34004, 32, 32)


Currently our labels are single integers that represent the class. However, for neural networks it's common to switch them to a so-called "one-hot" encoding, where each label is represented by a vector of length number of classes that has a one at the position  zeros everywhere else. 

For example, if there were 7 classes, we could represent that class 5 as `0000100`. In the case of the HASYv2 dataset, there's 369 unique classes, but we only picked the first 100, so each label should be a length 100 vector with a single one.

_Hint_ `keras` has a function for this

In [7]:
# Convert labels to one-hot encoding here
# TODO 
from keras.utils import to_categorical

y_train = to_categorical(y_train, 100)

print(y_train.shape) # Should be approx (34062, 100)

Using TensorFlow backend.


(34004, 100)


Next let's create a simple linear model using Keras to get ourselves familiar with it. Your model should consist of a `Flatten` layer that takes as input `(32, 32)` matrices and outputs a `1024` vector. Then, you should have a second `Dense` layer that has as many outputs as there are classes in the data (i.e 100), and has a "softmax" activation. Check out the [30 seconds to Keras](https://keras.io) intro for an introduction to the framework. If you're stuck, check the next blocks for some of the same ingredients.

In [66]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

# This function should return a keras Sequential model that has the appropriate layers
# TODO
def create_linear_model():
    model = Sequential()
    model.add(Flatten(input_shape=(32,32)))
    model.add(Dense(100, input_shape=(1024,), activation='softmax'))
    return model

model = create_linear_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               102500    
Total params: 102,500
Trainable params: 102,500
Non-trainable params: 0
_________________________________________________________________


Here we "compile" the model, specifying the loss function to use as well as the method for minimizing the loss function. In addition, we request that Keras provide us with constant accuracy measures. We also feed in the validation data, which means that Keras will evaluate the validation loss for us after each epoch (epoch generally means once it has seen each training sample once).  

In [69]:
# Feel free to try out other optimizers. Categorical crossentropy loss means 
# we are predicting the probability of each class separately.
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

model.fit(X_train, y_train, epochs=3, batch_size=64)

Epoch 1/12
34084/34084 [==============================] - 3s - loss: 15.7715 - acc: 0.0215     
Epoch 2/12
34084/34084 [==============================] - 3s - loss: 15.7715 - acc: 0.0215     
Epoch 3/12
34084/34084 [==============================] - 3s - loss: 15.7715 - acc: 0.0215     
Epoch 4/12
34084/34084 [==============================] - 4s - loss: 15.7715 - acc: 0.0215     
Epoch 5/12
34084/34084 [==============================] - 3s - loss: 15.7715 - acc: 0.0215     
Epoch 6/12
34084/34084 [==============================] - 2s - loss: 15.7715 - acc: 0.0215     
Epoch 7/12
34084/34084 [==============================] - 1s - loss: 15.7715 - acc: 0.0215     
Epoch 8/12
34084/34084 [==============================] - 1s - loss: 15.7715 - acc: 0.0215     
Epoch 9/12
34084/34084 [==============================] - 2s - loss: 15.7715 - acc: 0.0215     
Epoch 10/12
34084/34084 [==============================] - 2s - loss: 15.7715 - acc: 0.0215     
Epoch 11/12
34084/34084 [==============

In [70]:
y_test_copy = to_categorical(y_test.copy(), 100)
score, accuracy = model.evaluate(X_test, y_test_copy, batch_size=64, verbose=0)
print("Score : {},  Accuracy : {}%".format(score, accuracy*100))

Score : 15.775648403437888,  Accuracy : 2.1246458923512748%


The simple linear model probably didn't do too well. Let's create a CNN (Convolutional Neural Network) next. We've provided the code for the network, so just run these cells for now. Try to experiment here, adding and removing layers and tuning the hyperparameters to get better results on the validation data.

In [47]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from keras.backend import clear_session

def create_convolutional_model():
    model = Sequential()
    model.add(Conv2D(128, (3, 3), input_shape=(32, 32, 1))) # A convolutional layer
    model.add(MaxPooling2D((4,4))) # Max pooling reduces the complexity of the model
    model.add(Dropout(0.4)) # Randomly dropping connections within the network helps against overfitting
    model.add(Conv2D(128, (2, 2), activation="relu")) 
    model.add(BatchNormalization()) # Numbers within the network might get really big, so normalize them
    
    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dense(128, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dense(y_train.shape[1], activation="softmax"))

    return model

clear_session()

model = create_convolutional_model()
model.summary() # Get a summary of all the layers

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 128)       1280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 128)         65664     
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 6, 128)         512       
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1179904   
__________

In [48]:
y_val_copy = to_categorical(y_val.copy(), 100)

As you can see, our model has a lot of parameters. Optimizing this might take a while, depending on your PC.

In [49]:
# Feel free to try out other optimizers. Categorical crossentropy loss means 
# we are predicting the probability of each class separately.
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

# Extra axis for "gray" channel
model.fit(X_train[:, :, :, np.newaxis], y_train, epochs=10, batch_size=64, validation_data=(X_val[:, :, :, np.newaxis], y_val_copy))

Train on 34004 samples, validate on 4330 samples
Epoch 1/10
34004/34004 [==============================] - 128s - loss: 1.2078 - acc: 0.6789 - val_loss: 0.8325 - val_acc: 0.7277
Epoch 2/10
34004/34004 [==============================] - 107s - loss: 0.6042 - acc: 0.7995 - val_loss: 0.6553 - val_acc: 0.7861
Epoch 3/10
34004/34004 [==============================] - 103s - loss: 0.4746 - acc: 0.8338 - val_loss: 0.5895 - val_acc: 0.8044
Epoch 4/10
34004/34004 [==============================] - 104s - loss: 0.3949 - acc: 0.8563 - val_loss: 0.6321 - val_acc: 0.7924
Epoch 5/10
34004/34004 [==============================] - 135s - loss: 0.3385 - acc: 0.8702 - val_loss: 0.6033 - val_acc: 0.8016
Epoch 6/10
34004/34004 [==============================] - 128s - loss: 0.2912 - acc: 0.8878 - val_loss: 0.6153 - val_acc: 0.8118
Epoch 7/10
34004/34004 [==============================] - 95s - loss: 0.2528 - acc: 0.8995 - val_loss: 0.6160 - val_acc: 0.8176
Epoch 8/10
34004/34004 [=========================

Finally, let's see how well our model did on the held-out test data. This is basically what matters, after all. The second number should be test accuracy - you should be able to get approx 80% with our setup. Try to improve this, but be careful not to overfit on the test data. Always use the validation set to tune your model.

In [50]:
y_test_copy2 = to_categorical(y_test.copy(), 100)
score2, accuracy2 = model.evaluate(X_test[:, :, :, np.newaxis], y_test_copy2)
print("Score : {},  Accuracy : {}%".format(score2, accuracy2*100))

4295/4295 [==============================] - 5s     
Score : 0.652919534809792,  Accuracy : 81.4668218886894%
